In [35]:
# РАБОТА С ДАННЫМИ
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import gdown
from   sklearn.impute               import SimpleImputer
import subprocess
import shutil
import tempfile
import inspect # код методов preprocessing
from category_encoders import TargetEncoder
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)



# СЕТЬ
import torch
from   torch                        import nn
from   torch.utils.data             import DataLoader, Dataset, TensorDataset
import torch.optim as optim
import pytorch_lightning as pl
from   pytorch_lightning.callbacks  import ModelCheckpoint, EarlyStopping
from   sklearn.model_selection      import train_test_split
from   datetime                     import datetime
from   sklearn.preprocessing        import StandardScaler


# ЛОГИРОВАНИЕ:

# ОПИСАНИЕ
import os
from typing import Any, List, Optional, Union
import torchmetrics.regression as tm
from torchmetrics                   import MetricCollection
import torchmetrics
import joblib
import warnings
import time

# MLFLOW
from   pytorch_lightning.loggers    import MLFlowLogger
from   mlflow.tracking              import MlflowClient
import mlflow
from   pyngrok                      import ngrok
# from   google.colab                 import userdata

import sys
from IPython.display import Image

from playwright.sync_api import sync_playwright

def run_screenshot_script(url, output_path, width=1920, height=1480):
	"""
	Делает скриншот, запуская Playwright в изолированном подпроцессе,
	чтобы избежать конфликтов asyncio, не требуя отдельного файла скрипта.
	"""
	script_code = """
import sys
import time
from playwright.sync_api import sync_playwright

def take_screenshot(url, output_path, width, height):
	'''
	Запускает браузер, переходит по URL и сохраняет скриншот.
	'''
	try:
		with sync_playwright() as p:
			browser = p.chromium.launch()
			page = browser.new_page()
			page.set_viewport_size({"width": int(width), "height": int(height)})
			page.goto(url)
			time.sleep(5)  # Даем время на прогрузку JS
			page.screenshot(path=output_path)
			browser.close()
		print(f"Picture succesfully saved in {output_path}")
	except Exception as e:
		print(f"Error occured in Playwright subprocess: {e}", file=sys.stderr)
		sys.exit(1)

if __name__ == "__main__":
	script_url = sys.argv[1]
	script_output_path = sys.argv[2]
	script_width = sys.argv[3]
	script_height = sys.argv[4]

	take_screenshot(script_url, script_output_path, script_width, script_height)
"""
	python_executable = sys.executable
	command = [
		python_executable,
		"-c",
		script_code,
		url,
		output_path,
		str(width),
		str(height)
	]

	try:
		result = subprocess.run(
			command,
			capture_output=True,
			text=True,
			check=True
		)

		if result.stdout:
			print(result.stdout.strip())

		if result.stderr:
			print("Сообщения из потока ошибок:", result.stderr.strip(), file=sys.stderr)

	except subprocess.CalledProcessError as e:
		print("Ошибка: не удалось выполнить скрипт скриншота.", file=sys.stderr)
		print(f"Код возврата: {e.returncode}", file=sys.stderr)
		print(f"Stdout:\n{e.stdout}", file=sys.stderr)
		print(f"Stderr:\n{e.stderr}", file=sys.stderr)
	except FileNotFoundError:
		print(f"Ошибка: не удалось найти '{python_executable}'. Убедитесь, что Python установлен и доступен.", file=sys.stderr)

In [36]:
# #https://drive.google.com/file/d/1IANR8nX_HbbuCNaQ23lOVJQ6w2__H-xv/view?usp=sharing SAMPLE SUBMISSION
# #https://drive.google.com/file/d/1xfzes_aIviHAo0KKc505MiDb5s2kfQlT/view?usp=sharing TEST
# #https://drive.google.com/file/d/1EX_wQQwd1w-plOQvcPEVp10eeGUYFQBE/view?usp=sharing TRAIN
# SAMPLE_SUBMISSION = '1IANR8nX_HbbuCNaQ23lOVJQ6w2__H-xv'
# TEST              = '1xfzes_aIviHAo0KKc505MiDb5s2kfQlT'
# TRAIN             = '1EX_wQQwd1w-plOQvcPEVp10eeGUYFQBE'
# d = {'SAMPLE_SUBMISSION': SAMPLE_SUBMISSION,
#      'TEST': TEST,
#      'TRAIN': TRAIN}
# for name,id in d.items():
#   gdown.download(id=id, output= f'{name}.csv', quiet=False)

In [37]:
class PLModel(pl.LightningModule):
    def __init__(self, model, criterion, optimizer, metrics: dict = None, hyperparameters=None):
        super().__init__()
        self.save_hyperparameters(hyperparameters or {})
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

        metrics = metrics or {}

        # --- ИСПРАВЛЕНИЕ ЗДЕСЬ ---
        # 1. Создаем базовую коллекцию метрик.
        base_metrics = MetricCollection(metrics)

        # 2. Создаем независимые копии для train и val.
        self.train_metrics = base_metrics.clone()
        self.val_metrics = base_metrics.clone()

    def forward(self, x): return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        self.train_metrics.update(y_hat, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log_dict(self.train_metrics, on_step=False, on_epoch=True) # prog_bar=True можно убрать, если много метрик

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        self.val_metrics.update(y_hat, y)
        self.log('val_loss', loss, on_epoch=True)
        self.log_dict(self.val_metrics, on_step=False, on_epoch=True)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, _ = batch
        return self(x)

    def configure_optimizers(self): return self.optimizer

In [38]:
class DataModule(pl.LightningDataModule):
    """
    LightningDataModule для набора данных
    Инкапсулирует все шаги по загрузке и обработке данных.
    """
    def __init__(self,
                 y_label = 'price_per_month',
                 data_path: str = 'TRAIN.csv',
                 batch_size: int = 32,
                 cols_to_drop: list = []):
        super().__init__()
        self.data_path = data_path
        self.batch_size = batch_size
        self.encoder = TargetEncoder()
        self.scaler = StandardScaler()
        self.cols_to_drop = cols_to_drop
        self.y_label = y_label
        self.setup(None)

    def setup(self, stage: str):
        """
        Этот метод вызывается для каждого GPU/процесса.
        Здесь мы загружаем, преобразуем и разделяем данные.
        """
        df = pd.read_csv(self.data_path)
        
        df = df.drop(columns=[col for col in self.cols_to_drop if col in df.columns], errors='ignore')
        
        df[self.y_label] = np.log1p(df[self.y_label])

        X = df.drop(self.y_label, axis=1)
        y = df[self.y_label]

        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

        self.encoder.fit(X_train, y_train)
        X_train = self.encoder.transform(X_train)
        self.scaler.fit(X_train)
        X_train = self.scaler.transform(X_train)

        X_val = self.encoder.transform(X_val)
        X_val = self.scaler.transform(X_val)
        X_test = self.encoder.transform(X_test)
        X_test = self.scaler.transform(X_test)
        
        self.n_inputs = X_train.shape[1]

        if stage == 'fit' or stage is None:
            self.train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
            self.val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val.values, dtype=torch.float32))
        
        if stage == 'test' or stage is None:
            self.test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32))
        
        print("DataModule setup complete.")

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [39]:
if 'mlflow_process' in locals() and mlflow_process.poll() is None:
    mlflow_process.terminate()
    mlflow_process.wait()
    print("Остановлен предыдущий процесс MLflow UI.")

Остановлен предыдущий процесс MLflow UI.


In [40]:
port = 5000
mlflow_process = subprocess.Popen(["mlflow", "ui", "--port", str(port)])
print(f"MLflow UI запущен с PID (ID процесса): {mlflow_process.pid}")
mlflow_url = f"http://localhost:{port}"
mlflow.set_tracking_uri("file:./mlruns")
print(mlflow_url)

MLflow UI запущен с PID (ID процесса): 6840
http://localhost:5000


In [41]:
class ModelTemplate:
    """
    Класс-шаблон для динамического создания модели.
    Хранит 'тело' модели и собирает финальную архитектуру.
    Автоматически находит размерности входа и выхода тела,
    даже если оно начинается/заканчивается нелинейными слоями.
    """
    def __init__(self, body: torch.nn.Module):
        self.body = body
        self.body_in_features = None
        self.body_out_features = None

        # Ищем in_features, идя по слоям с начала
        for layer in self.body:
            if hasattr(layer, 'in_features'):
                self.body_in_features = layer.in_features
                break

        # Ищем out_features, идя по слоям с конца
        for layer in reversed(self.body):
            if hasattr(layer, 'out_features'):
                self.body_out_features = layer.out_features
                break

        # Проверка, что мы смогли найти нужные слои
        if self.body_in_features is None or self.body_out_features is None:
            raise ValueError(
                "Не удалось автоматически определить in_features или out_features из тела модели. "
                "Убедитесь, что 'тело' содержит хотя бы один слой с этими атрибутами (например, nn.Linear)."
            )

    def build(self, n_inputs: int, n_outputs: int) -> torch.nn.Sequential:
        """Собирает и возвращает финальную torch-модель."""
        # Создаем входной слой, который соединяется с телом
        input_layer = torch.nn.Linear(n_inputs, self.body_in_features)

        # Создаем выходной слой, который соединяется с телом
        output_layer = torch.nn.Linear(self.body_out_features, n_outputs)

        return torch.nn.Sequential(
            input_layer,
            *self.body, # Распаковываем все слои из тела
            output_layer
        )

In [42]:
train,test,sample_submission=pd.read_csv('TRAIN.csv'),pd.read_csv('TEST.csv'),pd.read_csv('SAMPLE_SUBMISSION.csv')


experiment_name = " Regression with Lightning"    
mlflow_logger = MLFlowLogger(
    experiment_name=experiment_name,
    tracking_uri="file:./mlruns"
)
    
data_module = DataModule(
    batch_size=32
    )


model_body = torch.nn.Sequential(
    torch.nn.Linear(256, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(256, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU()
)

MODEL_KWARGS={
    'lr': 0.01
    
}


model = PLModel(
    ModelTemplate(model_body).build(data_module.n_inputs,1),
    nn.MSELoss(),
    optim.Adam(),
    {
        'MAE': tm.MeanAbsoluteError(),
        'MSE': tm.MeanSquaredError()
},
    MODEL_KWARGS
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=True,
    mode='min',
    # min_delta = 1e-2
)
    
trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[early_stop_callback],
    logger=mlflow_logger
)
    
trainer.fit(model, datamodule=data_module)
trainer.test(model, datamodule=data_module)

run_id = trainer.logger.run_id
experiment_id = trainer.logger.experiment_id

run_url = f"http://localhost:{port}/#/experiments/{experiment_id}/runs/{run_id}"
screenshot_path = f"images/run_{run_url.split('/')[-1]}_metrics.png"

try:
    run_screenshot_script(run_url+ "/model-metrics", screenshot_path,1920,2200)
    display(Image(filename=screenshot_path))
except Exception as e:
    print(f"Failed to generate or display the screenshot: {e}")

mlflow.end_run()

DataModule setup complete.


TypeError: Adam.__init__() missing 1 required positional argument: 'params'